In [1]:
!pip install plotly.express --quiet

In [2]:
!pip install geobr --quiet

In [30]:
!pip install folium --quiet

In [2]:
import geopandas as gpd
import numpy as np
import plotly.express as px
import pandas as pd


In [3]:
ipea_df = pd.read_csv(r'C:\Users\cauai\Desktop\Projetos\ipeadataset\ipeadataset.csv',sep=';',skiprows=1)
ipea_df.head()

,Sigla,Código,Estado,1931,1932,1933,1934,1935,1936,1937,...,2012,2013,2014,2015,2016,2017,2018,2019,2020,2021
0,AC,12,Acre,360.00,324.96,220.2,144.0,138.0,150.0,120.0,...,1453,1411,2000,2229,2169,2629,1969,2057,2141,2490
1,AL,27,Alagoas,1296.00,1369.80,1500.0,1200.0,972.0,1200.0,1973.4,...,0,0,0,0,0,0,0,0,0,0
2,AM,13,Amazonas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2143,1993,429,2011,555,507,347,325,367,356
3,AP,16,Amapá,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0,0,0,0,0,0,0,0,0,0
4,BA,29,Bahia,24839.46,16025.58,12000.0,19296.0,15000.0,27120.0,16860.0,...,141902,158077,201715,209108,134824,160648,248605,180252,244259,210096


In [4]:
df_cafe = pd.melt(ipea_df,id_vars=['Sigla','Código','Estado'], var_name='Ano', value_name='Produção')

In [5]:
df_cafe.fillna(0,inplace=True)

In [6]:
df_cafe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2457 entries, 0 to 2456
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   Sigla     2457 non-null   object 
 1   Código    2457 non-null   int64  
 2   Estado    2457 non-null   object 
 3   Ano       2457 non-null   object 
 4   Produção  2457 non-null   float64
dtypes: float64(1), int64(1), object(3)
memory usage: 96.1+ KB


In [7]:
df_agg = df_cafe.groupby(['Sigla', 'Ano']).sum().reset_index()
df_agg

C:\Users\cauai\AppData\Local\Temp\ipykernel_17708\242312480.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



,Sigla,Ano,Código,Produção
0,AC,1931,12,360.00
1,AC,1932,12,324.96
2,AC,1933,12,220.20
3,AC,1934,12,144.00
4,AC,1935,12,138.00
...,...,...,...,...
2452,TO,2017,17,0.00
2453,TO,2018,17,0.00
2454,TO,2019,17,0.00
2455,TO,2020,17,0.00


In [8]:
df_agg

,Sigla,Ano,Código,Produção
0,AC,1931,12,360.00
1,AC,1932,12,324.96
2,AC,1933,12,220.20
3,AC,1934,12,144.00
4,AC,1935,12,138.00
...,...,...,...,...
2452,TO,2017,17,0.00
2453,TO,2018,17,0.00
2454,TO,2019,17,0.00
2455,TO,2020,17,0.00


In [9]:
esatdo_coords = {
    "AC": (-8.77, -70.55),
    "AL": (-9.71, -35.73),
    "AP": (1.41, -51.77),
    "AM": (-3.07, -61.66),
    "BA": (-12.96, -38.51),
    "CE": (-3.71, -38.54),
    "DF": (-15.83, -47.86),
    "ES": (-19.19, -40.34),
    "GO": (-16.64, -49.31),
    "MA": (-2.55, -44.30),
    "MT": (-12.64, -55.42),
    "MS": (-20.51, -54.54),
    "MG": (-18.10, -44.38),
    "PA": (-5.53, -52.29),
    "PB": (-7.06, -35.55),
    "PR": (-24.89, -51.55),
    "PE": (-8.28, -35.07),
    "PI": (-8.28, -43.68),
    "RJ": (-22.84, -43.15),
    "RN": (-5.22, -36.52),
    "RS": (-30.01, -51.22),
    "RO": (-10.83, -63.34),
    "RR": (1.99, -61.33),
    "SC": (-27.33, -49.44),
    "SP": (-23.55, -46.64),
    "SE": (-10.90, -37.07),
    "TO": (-10.25, -48.25)
}

In [10]:
estado_df = pd.DataFrame.from_dict(esatdo_coords, orient="index", columns=["latitude", "longitude"])
estado_df.index.name = "Sigla"

In [11]:
df_estado_prod = pd.merge(df_agg,estado_df,on='Sigla')


In [12]:
df_estado_prod_sort = df_estado_prod.sort_values('Ano')

In [15]:
fig = px.density_mapbox(df_estado_prod, lat='latitude', lon='longitude',z='Produção',radius=25, zoom=3,
                        color_continuous_scale=px.colors.sequential.YlOrRd,
                        mapbox_style="stamen-terrain")
fig.update_layout(
        title = 'Mapa de Calor - Produção de Histórica de Café',
)
fig.show()

In [14]:
import plotly.offline as pyo

# Save the figure as an HTML file
pyo.plot(fig, filename='coffee_production_map.html')

'coffee_production_map.html'

In [19]:
fig2 = px.bar(df_estado_prod, x='Sigla', y='Produção', color='Sigla', animation_frame='Ano')

# Set the title and axis labels
fig2.update_layout(
    title='Produção Anual de Café Anual por Estado',
    xaxis_title='Estado',
    yaxis_title='Produção (em toneladas)'
)

# Show the plot
fig2.show()


In [20]:

# Save the figure as an HTML file
pyo.plot(fig2, filename='Produção de Café Anual por Estado.html')

'Produção de Café Anual por Estado.html'